In [51]:
%matplotlib inline
import pandas as pd
import numpy as np
columns=['CountryCode', 'IndicatorCode', 'Year', 'Value']
data = pd.read_csv("data/indicators.csv", usecols=(columns))# nrows=1048576)

In [52]:
data.head()

,CountryCode,IndicatorCode,Year,Value
0,ARB,SP.ADO.TFRT,1960,1.335609e+02
1,ARB,SP.POP.DPND,1960,8.779760e+01
2,ARB,SP.POP.DPND.OL,1960,6.634579e+00
3,ARB,SP.POP.DPND.YG,1960,8.102333e+01
4,ARB,MS.MIL.XPRT.KD,1960,3.000000e+06


In [53]:
columns=['CountryCode']
countries = pd.read_csv("data/Country.csv", usecols=(columns))

In [54]:
countries.head()

,CountryCode
0,AFG
1,ALB
2,DZA
3,ASM
4,ADO


In [55]:
# find some measures that seem important and are often present, filter countries with all those intact, train network

In [56]:
columns=['LongDefinition', 'SeriesCode']
series = pd.read_csv("data/Series.csv", usecols=(columns))

In [57]:
series.head()

,SeriesCode,LongDefinition
0,BN.KLT.DINV.CD,Foreign direct investment are the net inflows ...
1,BX.KLT.DINV.WD.GD.ZS,Foreign direct investment are the net inflows ...
2,BX.KLT.DINV.CD.WD,Foreign direct investment refers to direct inv...
3,BM.KLT.DINV.GD.ZS,Foreign direct investment are the net inflows ...
4,BN.TRF.KOGT.CD,Net capital account records acquisitions and d...


In [58]:
print (len(countries))
print (len(series))

247
1345


In [59]:
#isUsedSeries = [(name in seriesToUse) for name in series.SeriesCode.values]
#filteredSeries = series[isUsedSeries]
#filteredSeriesNames = filteredSeries.SeriesCode.values

#useableData = [(name in filteredSeriesNames) for name in data.IndicatorCode.values]
#filteredData = data[useableData]
seriesCounts = data['IndicatorCode'].value_counts()

a = seriesCounts.to_dict()
useableData = [(a.get(name, 0) > 10000) for name in data.IndicatorCode.values]
filteredData = data[useableData]

In [60]:
useableSeries = [(a.get(name, 0) > 10000) for name in series.SeriesCode.values]
filteredSeries = series[useableSeries].SeriesCode.values

In [61]:
print(len(filteredData))
print(len(data))
print(len(filteredSeries))
print(len(series))

732948
5656458
63
1345


In [62]:
# a = data[(data.CountryCode == 'ARB') & ((data.IndicatorCode == 'SP.ADO.TFRT') | (data.IndicatorCode == 'SP.POP.DPND'))]
useableCountries = []
for country in countries.CountryCode.values:
    useable = True
    filterByCountry = filteredData[(data.CountryCode == country)]
    for seriesName in filteredSeries:
        if len(filterByCountry[(filterByCountry.IndicatorCode == seriesName)]) == 0:
            useable = False;
            break;
    if useable:
        useableCountries.append(country)

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [63]:
len(useableCountries)

175

In [64]:
goodData = [(country in useableCountries) for country in filteredData.CountryCode.values]
useableData = filteredData[goodData]

In [65]:
print(len(useableData))
useableData.head()

559891


,CountryCode,IndicatorCode,Year,Value
350,EAP,SP.ADO.TFRT,1960,75.043631
351,EAP,SP.POP.DPND,1960,78.574185
352,EAP,SP.POP.DPND.OL,1960,6.480808
353,EAP,SP.POP.DPND.YG,1960,71.976235
359,EAP,SP.DYN.CBRT.IN,1960,26.635950


In [66]:
# Filtered Data reasonably above, need to:
# Seperate into few year blocks, with next year values as results to guess
# Train a few different models to guess next years values

In [68]:
predictionFeature = 'SP.ADO.TFRT'
yearsToPredict = 5
predictionData = filteredData[(filteredData.IndicatorCode == predictionFeature)]

trainingValues = []
trainingResults = []
for country in useableCountries:#[0:20]: # 213
    countryData = filteredData[(filteredData.CountryCode == country)]
    predictionCountryData = predictionData[(predictionData.CountryCode == country)]
    pastData_5 = []
    pastData_4 = []
    pastData_3 = []
    pastData_2 = []
    pastData_1 = []
    for year in range(1960, 2010): # 1960
        pastData_5 = pastData_4
        pastData_4 = pastData_3
        pastData_3 = pastData_2
        pastData_2 = pastData_1
        
        yearData = countryData[(countryData.Year == year)]
        
        pastData_1 = []
        for series in filteredSeries:#[0:30]: # 40
            # TODO: see if I can get rid of this loop
            seriesData = yearData[(yearData.IndicatorCode == series)].Value.values
            if len(seriesData) != 1:
                pastData_5 = []
                pastData_4 = []
                pastData_3 = []
                pastData_2 = []
                pastData_1 = []
                break;
            value = seriesData[0]
            #print (value)
            pastData_1.append(value)
            
        predictionValues = []
        for i in range(yearsToPredict):
            predictionValues.extend(predictionCountryData[(countryData.Year == year + 1 + i)].Value.values)
            
        valid = (len(predictionValues) == yearsToPredict) & (len(pastData_5) != 0) & (len(pastData_4) != 0) & (len(pastData_3) != 0) & (len(pastData_2) != 0) & (len(pastData_1) != 0)
        if valid:
            trainingValues.append([country + '_' + str(year)] + pastData_5 + pastData_4 + pastData_3 + pastData_2 + pastData_1)
            trainingResults.append(predictionValues)

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:40: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [72]:
xColumns = ['OBSERVATION'] + [series + '_' + str(i) for i in range(yearsToPredict) for series in filteredSeries]
xData = pd.DataFrame(trainingValues, columns=xColumns)
yColumns = ['YEAR_' + str(i) for i in range(yearsToPredict)]
yData = pd.DataFrame(data=trainingResults, columns=yColumns)
print (len(xData))
print (len(yData))

5147
5147


In [73]:
xData.head()

,OBSERVATION,NY.GDP.MKTP.CD_0,NY.GDP.PCAP.CD_0,AG.PRD.CREL.MT_0,AG.YLD.CREL.KG_0,AG.PRD.CROP.XD_0,AG.PRD.FOOD.XD_0,AG.LND.CREL.HA_0,AG.PRD.LVSK.XD_0,EN.POP.DNST_0,...,SP.DYN.TFRT.IN_4,TX.VAL.MRCH.CD.WT_4,TX.VAL.MRCH.WL.CD_4,TX.VAL.MRCH.RS.ZS_4,TX.VAL.MRCH.HI.ZS_4,TM.VAL.MRCH.CD.WT_4,TM.VAL.MRCH.WL.CD_4,TM.VAL.MRCH.RS.ZS_4,TM.VAL.MRCH.HI.ZS_4,TG.VAL.TOTL.GD.ZS_4
0,AFG_1971,1.673333e+09,161.384702,3826000.0,1224.516,83.05,70.20,3124500.0,60.38,15.881812,...,7.450,100000000.0,99900000.0,47.314314,42.712713,141000000.0,145386000.0,32.425405,57.253106,13.161423
1,AFG_1977,1.733333e+09,144.173944,4270000.0,1279.592,91.25,73.56,3337000.0,58.45,18.415149,...,7.449,306000000.0,313137000.0,45.115397,35.590492,328000000.0,458652000.0,45.553055,45.343964,21.467268
2,AFG_1978,2.155555e+09,175.027098,4351000.0,1301.915,94.92,76.02,3342000.0,61.00,18.864003,...,7.449,321000000.0,269156000.0,43.718141,27.069432,395000000.0,450516000.0,53.467801,34.542835,21.696969
3,AFG_1979,2.366667e+09,188.085136,4481000.0,1316.392,97.68,78.24,3404000.0,63.38,19.273587,...,7.449,474000000.0,435015000.0,60.016551,18.152248,425000000.0,551216000.0,68.909647,24.561515,24.310830
4,AFG_1980,2.555556e+09,199.164809,4624000.0,1362.404,102.93,83.72,3394000.0,69.07,19.654077,...,7.449,670000000.0,670474000.0,64.592065,19.471896,841000000.0,494754000.0,60.669343,32.162044,41.491344


In [74]:
yData.head()

,YEAR_0,YEAR_1,YEAR_2,YEAR_3,YEAR_4
0,145.3210,146.2150,147.1090,148.0030,148.8970
1,150.8782,151.9654,153.0526,154.1398,155.2270
2,151.9654,153.0526,154.1398,155.2270,156.3758
3,153.0526,154.1398,155.2270,156.3758,157.5246
4,154.1398,155.2270,156.3758,157.5246,158.6734


In [75]:
xData.to_csv("data/xSave.csv")
yData.to_csv("data/ySave.csv")